In [ ]:
# Compare the measurement values of a certain parameter at different sites near the MINEWATER site in the same year and month
import pandas as pd
import matplotlib.pyplot as plt

file_path = 'D:\\PycharmProjects\\minewaters_new\\distancesS71567.csv'
df = pd.read_csv(file_path, low_memory=False)

# Filter out the data for the year 2018 and records where parameter_name is 'pH'
df_filtered = df[(df['sampling_year'] == 2018) & (df['parameter_name'] == 'pH')]

# Convert the sampling_datetime column to date format and extract year and month
df_filtered['sampling_datetime'] = pd.to_datetime(df_filtered['sampling_datetime'])
df_filtered['month'] = df_filtered['sampling_datetime'].dt.to_period('M')

# Group by month and calculate the average measurement value for each station each month
grouped = df_filtered.groupby('month')[['station_number', 'sample_value', 'distance']]

# Sort the data for each month by distance
# Use the apply method of the groupby object, ensuring that the return is a DataFrame
monthly_data = grouped.apply(lambda x: x.sort_values(by='distance')).reset_index()

# Get the grouping key, which is each month
months = monthly_data['month'].unique()

# Generate a bar chart for each month
for month in months:
    # Filter the data for the current month
    month_data = monthly_data[monthly_data['month'] == month]

    # Create a bar chart
    plt.figure(figsize=(10, 8))
    # Ensure month_data is a DataFrame and access data using column names
    plt.bar(month_data['station_number'], month_data['sample_value'], color='blue')

    # Set the title and axis labels
    plt.title(f'Monthly Solids, Suspended at 105 C - {month}')
    plt.xlabel('Station Number')
    plt.ylabel('Sample Value (Solids, Suspended at 105 C)')

    # Rotate x-axis labels to prevent overlap
    # plt.xticks(rotation=45, labels=month_data['station_number'].astype(str))

    # Save the figure
    output_filename = f'D:\\PycharmProjects\\minewaters_new\\pic\\Solids, Suspended at 105 C_{month.strftime("%Y-%m")}.png'
    plt.savefig(output_filename)
    print(f'Saved figure to {output_filename}')

    # Close the figure to free memory
    plt.close()

print("Bar charts for each month have been generated and saved.")


In [ ]:
# Draw a map of MINEWATER stations and their 10 nearest stations for each year
import os
import pandas as pd
import folium
from sklearn.neighbors import NearestNeighbors
from pyproj import Transformer

# Define data directory and target year
data_dir = r'D:\PycharmProjects\minewaters_new\CleanYear'
year = 2022

# Create a coordinate transformer
transformer = Transformer.from_crs('epsg:27700', 'epsg:4326', always_xy=True)

# Create a function to convert coordinates
def convert_coordinates(easting, northing):
    lon, lat = transformer.transform(easting, northing)
    return lon, lat

# Define a list of available colors
folium_colors = ['red', 'blue', 'green', 'purple', 'orange', 'darkred', 'lightred',
                 'beige', 'darkblue', 'darkgreen', 'cadetblue', 'darkpurple', 'white',
                 'pink', 'lightblue', 'lightgreen', 'gray', 'black', 'lightgray']

# Initialize the map and set the initial center
map_center = [53.0, -2.0]  # Set a suitable center point
m = folium.Map(location=map_center, zoom_start=6)

# Load the dataset
file_path = os.path.join(data_dir, f'{year} Water Quality Archive Cleaned.csv')
if not os.path.exists(file_path):
    print(f"File not found: {file_path}")
else:
    data = pd.read_csv(file_path, low_memory=False)

    data = data.drop_duplicates(subset='station_number', keep='first')

    data[['longitude', 'latitude']] = data.apply(lambda row: convert_coordinates(row['easting'], row['northing']), axis=1, result_type='expand')

    # Filter for stations of type 'MINEWATER'
    minewaters_data = data[data['station_type'] == 'MINEWATER']
    unique_stations = data.drop_duplicates(subset=['station_number'])
    coordinates = unique_stations[['longitude', 'latitude']].values

    # Use NearestNeighbors to find the nearest stations
    n_neighbors = min(11, len(unique_stations))
    nbrs = NearestNeighbors(n_neighbors=n_neighbors, algorithm='ball_tree').fit(coordinates)

    # Create a FeatureGroup and set it to not display by default
    fg = folium.FeatureGroup(name=str(year), show=False)

    # Iterate through each 'MINEWATER' station
    for idx, (row_idx, row) in enumerate(minewaters_data.iterrows()):
        station_id = row['station_number']

        station_index = unique_stations[unique_stations['station_number'] == station_id].index[0]

        indices = nbrs.kneighbors([row[['longitude', 'latitude']].values], return_distance=False)

        # Get neighbor stations
        neighbor_indices = indices.flatten()[1:min(n_neighbors, len(indices.flatten()))]
        neighbor_data = unique_stations.iloc[neighbor_indices].copy()
        neighbor_station_numbers = neighbor_data['station_number'].tolist()
        neighbor_station_numbers_str = ", ".join(map(str, neighbor_station_numbers))

        # Use the colors provided by Folium
        color = folium_colors[idx % len(folium_colors)]

        # Add the 'MINEWATER' station to the FeatureGroup
        folium.Marker(
            location=[row['latitude'], row['longitude']],
            popup=folium.Popup(f'MINEWATER Station {station_id} ({year})<br>Neighbors: {neighbor_station_numbers_str}', parse_html=True),
            icon=folium.Icon(color=color, icon='star')
        ).add_to(fg)

        # Draw neighbor stations
        for _, neighbor_row in neighbor_data.iterrows():
            neighbor_station_id = neighbor_row['station_number']
            folium.CircleMarker(
                location=[neighbor_row['latitude'], neighbor_row['longitude']],
                radius=5,
                popup=folium.Popup(f'Station {neighbor_station_id} ({year})', parse_html=True),
                color=color,
                fill=True,
                fill_color=color,
                fill_opacity=0.6
            ).add_to(fg)

    # Add the FeatureGroup to the map
    fg.add_to(m)

    # Add layer control
    folium.LayerControl().add_to(m)

    # Save the map to a folder
    output_folder = os.path.join(data_dir, 'MINEWATER_Maps')
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    map_path = os.path.join(output_folder, 'minewater_stations_map.html')
    m.save(map_path)

    print(f"Interactive map saved successfully in folder: {output_folder}")


In [ ]:
# Count the years each MINEWATER station appeared from 2000 to 2023
import pandas as pd
import os

# Result storage dictionary, key is station_number, value is a list of years appeared
station_years = {}

# Iterate from 2000 to 2023
for year in range(2000, 2024):
    filename = f'D:\\PycharmProjects\\minewaters_new\\convert\\{year} Water Quality Archive Unified.csv'
    if os.path.exists(filename):  # Check if the file exists
        df = pd.read_csv(filename, low_memory=False)
        # Filter MINEWATER stations
        minewater_df = df[df['station_type'] == 'MINEWATER']

        # Iterate through the filtered DataFrame and update station appearance years
        for _, row in minewater_df.iterrows():
            station_number = row['station_number']
            if station_number in station_years:
                if year not in station_years[station_number]:
                    station_years[station_number].append(year)
            else:
                station_years[station_number] = [year]

# Convert the result storage dictionary to a DataFrame
result_df = pd.DataFrame(list(station_years.items()), columns=['station_number', 'years'])

# Output the results to a CSV file
output_file = 'D:\\PycharmProjects\\minewaters_new\\MINEWATER_Station_Occurrences_Years.csv'
result_df.to_csv(output_file, index=False)

print(f"The occurrences and years of each MINEWATER station have been saved to {output_file}")

In [ ]:
# Find all 10 closest stations to a specific MINEWATER station from 2000 to 2024
import pandas as pd
from scipy.spatial.distance import euclidean

# Define a function to calculate the Euclidean distance between two points
def calculate_distance(easting1, northing1, easting2, northing2):
    return euclidean((easting1, northing1), (easting2, northing2))

# Initialize an empty DataFrame to store the final results
final_result = pd.DataFrame()

# Iterate through files from 2000 to 2023
for year in range(2000, 2024):
    filename = f'D:\\PycharmProjects\\minewaters_new\\convert\\{year} Water Quality Archive Unified.csv'

    # Read the CSV file
    df = pd.read_csv(filename, low_memory=False)

    # Remove duplicates based on station_name, keeping the first occurrence
    df_unique = df.drop_duplicates(subset='station_name', keep='first')

    # Filter for the station_number 'S71567'
    S71567_df = df_unique[df_unique['station_number'] == 'S71567']

    if not S71567_df.empty:
        # Get the coordinates of the S71567 station
        S71567_row = S71567_df.iloc[0]
        S71567_easting, S71567_northing = S71567_row['easting'], S71567_row['northing']

        # Calculate the distance of each station in df_unique to the S71567 station, and add it to a new column
        df_unique['distance_to_S71567'] = df_unique.apply(
            lambda row: calculate_distance(row['easting'], row['northing'], S71567_easting, S71567_northing), axis=1
        )

        # Sort by distance and select the index of S71567 station and its nearest 10 stations
        sorted_df = df_unique.sort_values('distance_to_S71567')
        nearest_indices = sorted_df.index[:11]  # Includes S71567 station and the nearest 10 stations

        # Use the station_numbers in the list as indexes to get all corresponding rows in df
        for index in nearest_indices:
            station_number = df_unique.loc[index, 'station_number']
            # Find all rows in the original df with the same station_number
            same_station_number_rows = df[df['station_number'] == station_number]
            final_result = pd.concat([final_result, same_station_number_rows], ignore_index=True)

    # If the current year's file does not have the S71567 station, print a message
    else:
        print(f"Year {year}: No station with station_number 'S71567' found.")

# Save the results to a new CSV file
output_filename = 'D:\\PycharmProjects\\minewaters_new\\Closest_Stations_to_S71567.csv'
final_result.to_csv(output_filename, index=False, encoding='utf-8-sig')

print(f"The result has been written to {output_filename}")

In [ ]:
# Parameter line chart visualization
import pandas as pd
import matplotlib.pyplot as plt

# Read the CSV file
file_path = 'D:\\PycharmProjects\\minewaters_new\\NEATH.csv'
df = pd.read_csv(file_path)

# Define the range of years to analyze and the parameter
years_to_include = range(2000, 2024)  
parameter_to_plot = 'Magnesium,Dissolved'  

# Filter data for the specified years and parameter
filtered_df = df[(df['sampling_year'].isin(years_to_include)) & (df['parameter_name'] == parameter_to_plot)]

# Ensure sampling_datetime is in datetime format
filtered_df['sampling_datetime'] = pd.to_datetime(filtered_df['sampling_datetime'], errors='coerce')

# Remove rows with conversion errors
filtered_df = filtered_df.dropna(subset=['sampling_datetime'])

# Extract year and month, and create a complete time column
filtered_df['YearMonth'] = filtered_df['sampling_datetime'].dt.to_period('M').astype(str)

# Group by station, year, and month, and calculate the monthly average
monthly_avg = filtered_df.groupby(['station_number', 'station_type', 'YearMonth'])['sample_value'].mean().reset_index()

# Convert YearMonth to datetime format to ensure correct sorting
monthly_avg['YearMonth'] = pd.to_datetime(monthly_avg['YearMonth'], format='%Y-%m')

# Sort by time
monthly_avg = monthly_avg.sort_values(by='YearMonth')

# Define colors for each station
station_colors = {
    'S78396': 'blue',
    'S10003': 'green',
    'S10004': 'red',
    'S78410': 'purple'
    # Add more colors according to your actual station names
}

# Plot the line chart
plt.figure(figsize=(12, 8))

# Plot the line chart for each station
for station in monthly_avg['station_number'].unique():
    station_data = monthly_avg[monthly_avg['station_number'] == station]
    station_type = station_data['station_type'].iloc[0]  
    color = station_colors.get(station, 'black')  
    plt.plot(station_data['YearMonth'], station_data['sample_value'], marker='o', color=color, label=f'Station {station} ({station_type})')

# Add legend, title, and labels
plt.legend()
plt.title(f'Monthly Average of {parameter_to_plot} from {years_to_include.start} to {years_to_include.stop-1}')
plt.xlabel('Year-Month')
plt.ylabel('Average Sample Value')
plt.xticks(rotation=45)
plt.grid(True)

# Show the plot
plt.tight_layout()
plt.show()

In [ ]:
# Draw heatmaps for various parameters
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns

# Step 1: Read the CSV file
file_path = 'D:\\PycharmProjects\\minewaters_new\\distancesS71567.csv' 
data = pd.read_csv(file_path)

# Step 2: Data transformation
# Bin the distance into 5 categories
data['distance_bin'] = pd.qcut(data['distance'], q=5, labels=['Near', 'Close', 'Mid', 'Far', 'Distant'])

# Calculate the average measurement value for each parameter in each distance bin
average_values = data.groupby(['distance_bin', 'parameter_name'])['sample_value'].mean().reset_index()

# Step 3: Draw the heatmap
# Rearrange the data using pivot
pivot_table = average_values.pivot(index='parameter_name', columns='distance_bin', values='sample_value')

# Draw the heatmap
plt.figure(figsize=(10, 8))  
sns.heatmap(pivot_table,
            annot=True,  
            fmt=".2f",  
            cmap='coolwarm',  
            cbar=True)  

# Adjust the heatmap
plt.title('Heatmap of Average Sample Values by Distance Bins and Parameters')
plt.xlabel('Distance Bin')
plt.ylabel('Parameter Name')
plt.show()

In [ ]:
# Draw a map for specified stations
import pandas as pd
import folium
import os
os.environ['USE_PYGEOS'] = '0'
import geopandas as gpd
from shapely.geometry import shape
from pyproj import Transformer

# Create a coordinate transformer
transformer = Transformer.from_crs('epsg:27700', 'epsg:4326', always_xy=True)

# Function to convert coordinates
def convert_coordinates(easting, northing):
    lon, lat = transformer.transform(easting, northing)
    return lon, lat

# Define data directory and target year
data_dir = r'D:\PycharmProjects\minewaters_new\CleanYear'
year = 2022

# Specify the station numbers to output
specific_stations = ['S10003', 'S10004', 'S10008', 'S10009', 'S71586', 'S71625', 'S71660', 'S71661', 'S71662', 'S71663',
                     'S71730', 'S76600', 'S71571', 'S10010', 'S51343', 'S78415',  'S71539', 'S78410', 'S19142','S78396']

folium_colors = ['red', 'blue', 'green', 'purple', 'orange', 'darkred', 'lightred',
                 'beige', 'darkblue', 'darkgreen', 'cadetblue', 'darkpurple',
                 'pink', 'lightblue',  'black']

# Initialize the map and set the initial center
map_center = [53.0, -2.0]  # Set a suitable center point
m = folium.Map(location=map_center, zoom_start=6)

# Load the dataset
file_path = 'D:\\PycharmProjects\\minewaters_new\\total_station_info.csv'
if not os.path.exists(file_path):
    print(f"File not found: {file_path}")
else:
    # Load the dataset
    data = pd.read_csv(file_path, low_memory=False)

    # Remove duplicate stations with the same 'station_number'
    data = data.drop_duplicates(subset='station_number', keep='first')

    # Apply coordinate transformation to the data
    data[['longitude', 'latitude']] = data.apply(lambda row: convert_coordinates(row['easting'], row['northing']),
                                                 axis=1, result_type='expand')

    # Filter only the specified stations
    filtered_data = data[data['station_number'].isin(specific_stations)]

    # Create a FeatureGroup and set it to not display by default
    fg = folium.FeatureGroup(name=str(year), show=False)

    # Iterate through each station
    for idx, (row_idx, row) in enumerate(filtered_data.iterrows()):
        station_id = row['station_number']

        # Use colors provided by folium
        color = folium_colors[idx % len(folium_colors)]

        # Add circle markers
        folium.CircleMarker(
            location=[row['latitude'], row['longitude']],
            radius=5,  # Adjust the radius of the circle
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.6,
            tooltip=f'Station {station_id}',  # Tooltip displays the station number
        ).add_to(fg)

        # Add the station number, adjusting position to avoid overlap
        folium.Marker(
            location=[row['latitude'], row['longitude']],
            icon=folium.DivIcon(
                html=f'<div style="font-size: 12px; color: {color}; font-weight: bold; transform: translate(-50%, -50%);">{station_id}</div>'
            ),
            icon_anchor=(0, 0)  # Adjust the anchor point of the icon
        ).add_to(fg)

    # Add the FeatureGroup to the map
    fg.add_to(m)

    # Load Neath River data (GeoJSON or Shapefile)
    neath_river_data_path = r'D:\PycharmProjects\minewaters_new\export.geojson'
    neath_river = gpd.read_file(neath_river_data_path)

    # Draw Neath River on the map
    for _, row in neath_river.iterrows():
        geometry = shape(row['geometry'])
        # Convert to longitude and latitude
        if geometry.geom_type == 'Polygon':
            coords = list(geometry.exterior.coords)
        elif geometry.geom_type == 'LineString':
            coords = list(geometry.coords)
        else:
            continue

        folium.PolyLine(locations=[(lat, lon) for lon, lat in coords],
                        color='blue',
                        weight=2.5,
                        opacity=1).add_to(m)

    # Add layer control
    folium.LayerControl().add_to(m)

    # Save the map to a folder
    output_folder = os.path.join(data_dir, 'MINEWATER_Maps')
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    map_path = os.path.join(output_folder, 'NEATH2.html')
    m.save(map_path)

    print(f"Interactive map saved successfully in folder: {output_folder}")


In [ ]:
# Build a new dataset categorized by local_authority
import os
import pandas as pd

# Define the path to the CSV file
summary_csv_path = 'D:\\PycharmProjects\\minewaters_new\\unique_values_summary.csv'
archive_folder_path = 'D:\\PycharmProjects\\minewaters_new\\convert'

# Read the summary CSV file
summary_df = pd.read_csv(summary_csv_path)

# Create a dictionary to store data corresponding to each local_authority
la_data_dict = {}

# Iterate over the years from 2000 to 2023
for year in range(2000, 2024):
    # Construct the full path for the archive file
    archive_csv_path = os.path.join(archive_folder_path, f'{year} Water Quality Archive Unified.csv')

    # Read the archive file
    if os.path.exists(archive_csv_path):
        archive_df = pd.read_csv(archive_csv_path, low_memory=False)

        # Iterate over local_authority in the summary CSV
        for la in summary_df['local_authority'].unique():
            # Filter the archive file's data based on local_authority
            filtered_df = archive_df[archive_df['local_authority'] == la]

            # If the filtered data is not empty, add it to the corresponding local_authority key in the dictionary
            if not filtered_df.empty:
                if la not in la_data_dict:
                    la_data_dict[la] = filtered_df
                else:
                    la_data_dict[la] = pd.concat([la_data_dict[la], filtered_df])

# Save the data in the dictionary to CSV files
new_path = 'D:\\PycharmProjects\\minewaters_new\\cluster'
for la, data in la_data_dict.items():
    new_csv_path = os.path.join(new_path, f'{la}.csv')
    data.to_csv(new_csv_path, index=False)
    print(f'File saved: {new_csv_path}')

In [ ]:
import pandas as pd
from scipy.spatial import ConvexHull
import matplotlib.pyplot as plt
import matplotlib.cm as cm

# Read the data
all_data = pd.read_csv('D:\\PycharmProjects\\minewaters_new\\convert\\All_Station_Data.csv', low_memory=False)

# Remove rows where local_authority is blank or 'UNKNOWN'
all_data = all_data[all_data['local_authority'].str.strip().notna() & (all_data['local_authority'] != 'UNKNOWN')]

# Create a figure
plt.figure(figsize=(12, 8))

# Group by local_authority
grouped = all_data.groupby('local_authority')

# Generate a unique color for each local_authority
num_colors = len(grouped)
colors = cm.get_cmap('hsv', num_colors) 

# Calculate and plot the convex hull for each local_authority
for i, (local_auth, group) in enumerate(grouped):
    try:
        # Calculate the convex hull
        hull = ConvexHull(group[['easting', 'northing']])

        # Generate color
        color = colors(i / num_colors)

        # Plot stations
        plt.scatter(group['easting'], group['northing'], alpha=0.7, label=f'{local_auth.strip()} stations', color=color)

        # Plot the convex hull boundary
        for simplex in hull.simplices:
            points = group.iloc[simplex][['easting', 'northing']].values
            plt.plot([points[0, 0], points[1, 0]], [points[0, 1], points[1, 1]], color=color, linewidth=1)

        # Close the convex hull boundary
        for simplex in hull.simplices:
            points = group.iloc[simplex][['easting', 'northing']].values
            # Add last segment to close the polygon
            plt.plot([points[-1, 0], points[0, 0]], [points[-1, 1], points[0, 1]], color=color, linewidth=1)

    except RuntimeError as e:  # Catch RuntimeError to handle QhullError
        print(f"Error occurred while processing {local_auth}: {e}")

# Set legend and adjust its position
plt.legend(loc='best', bbox_to_anchor=(1, 1)) 

# Set the chart title and axis labels
plt.title('Convex Hulls of All Local Authorities')
plt.xlabel('Easting')
plt.ylabel('Northing')

plt.grid(True)
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import numpy as np

# Read the CSV file
df = pd.read_csv('D:\\PycharmProjects\\minewaters_new\\distancesS71567.csv', low_memory=False)

# Ensure 'sampling_datetime' column is in datetime format
df['sampling_datetime'] = pd.to_datetime(df['sampling_datetime'])

# Specify the parameters to analyze
parameter_names = ['Solids, Suspended at 105 C','Temperature of Water','Phosphate :- {TIP}','pH','pH : In Situ','Oxygen, Dissolved as O2','Oxygen, Dissolved, % Saturation','Orthophosphate, reactive as P','Nitrogen, Total Oxidised as N','Nitrite as N','Discharge occurrence : In Situ','Ammoniacal Nitrogen as N','Copper','Zinc','Copper, Dissolved','Chemical Oxygen Demand :- {COD}', 'Conductivity at 25 C', 'Conductivity at 20 C']

# Create a color map
cmap = plt.get_cmap('nipy_spectral')

# Iterate over each parameter name
for parameter in parameter_names:
    # Filter rows for the current parameter name
    df_parameter = df[df['parameter_name'] == parameter]

    # Create a figure and axis
    fig, ax = plt.subplots(figsize=(12, 8))

    # Get all stations and distances
    stations_distance = df_parameter[['station_number', 'distance']].drop_duplicates()
    stations_distance = stations_distance.sort_values(by='distance')

    # Assign different colors to each station
    colors = cmap(np.linspace(0, 1, len(stations_distance)))

    # Store legend labels and corresponding colors
    handles = []

    # Iterate over each station's group and plot its values
    for color, (station, group) in zip(colors, stations_distance.groupby('station_number')):
        group_sorted = df_parameter[df_parameter['station_number'] == station].sort_values(by='sampling_datetime')
        line, = ax.plot(group_sorted['sampling_datetime'], group_sorted['sample_value'],
                        label=f'Station {station} - Distance: {group["distance"].iloc[0]:.2f}',
                        color=color)
        handles.append(line)

    # Set the chart title and labels
    ax.set_title(f'{parameter} Over Time for All Stations')
    ax.set_xlabel('Sampling Datetime')
    ax.set_ylabel(parameter)

    # Show grid
    plt.grid(True)

    # Sort the legend by distance
    sorted_labels = sorted(handles, key=lambda h: float(h.get_label().split('Distance: ')[1].split(' ')[0]))
    ax.legend(handles=sorted_labels, title='Stations (Distance)')

    # Show the figure
    plt.show()
